In [1]:
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import *
from ase.lattice.surface import surface
from ase import Atoms
from ase import io
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
from ase.io.cp2k import iread_cp2k_dcd
from ase.io.cp2k import read_cp2k_dcd
from ase import neighborlist
from ase.build import molecule
from scipy import sparse
from ase.io.trajectory import write_traj

/home/ahlawat/.local/lib/python3.9/site-packages/ase/utils/geometry.py:8: UserWarning: Moved to ase.geometry and ase.build
  warnings.warn('Moved to ase.geometry and ase.build')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/surface.py:17: UserWarning: Moved to ase.build
  warnings.warn('Moved to ase.build')


# convert CP2K DCD trajectory and xyz forces file to ACE/MACE training format

In [137]:
import mdtraj as md
traj = md.load_dcd('2H_SCAN_NPT-pos-1.dcd', top='slab_24.pdb')
topology = traj.topology
cell = []
step = []
forces = []
energies = []
num_atoms = 288
with open("2H_SCAN_NPT-FORCES-frc-1.xyz", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if str('   ')+str(num_atoms) not in line and not 'time' in line:
            forces.append(line.strip())
        if 'time' in line:
            while True:
                txt = line.split(',')[2]
                energy = txt.split()[2]
                energies.append(energy)
                break

In [139]:
len(forces)/288

17734.0

In [140]:
len(energies)

17734

In [125]:
17734*288

5107392

In [141]:
with open("training.xyz",'w') as f:
    i = 0
    j = 0
    k = 0
    for force in forces:
        if (i%288==0):
            k = 0
            a = traj.unitcell_vectors[j]
            b = np.asarray(a).reshape(-1)
            energy = energies[j]
            header = 'Lattice="{0:.4f} {1:.4f} {2:.4f} {3:.4f} {4:.4f}, {5:.4f} {6:.4f} {7:.4f} {8:.4f}"' \
                     .format(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]) \
                     + ' ' + 'Properties=species:S:1:pos:R:3:force:R:3' + '  ' \
                     + 'virial="0 0 0 0 0 0 0 0 0"' + '  '  \
                     + 'energy={}'.format(energy) + ' ' + 'config_type=2H_300 pbc="T T T"'
            f.write("{:}\n".format(header))
            frame = traj.xyz[j]
            j = j + 1 
        x, y, z = frame[k]*10
        species_name = (topology.atom(k)).name
        f.write("{}\t".format(species_name))
        f.write("{:12.8f}\t  {:12.8f}\t {:12.8f}\t".format(x,y,z))
        f.write("{}\n".format(force[3:63]))
        i = i + 1
        k = k + 1

In [ ]:
# import mdtraj as md
# traj = md.load_dcd('2H_SCAN_NPT-pos-1.dcd', top='slab_24.pdb')
# myfile = open("output.xyz","w")
# num_frames = len(traj.xyz)
# for i in range(num_frames):
#     frame = traj.xyz[i]
#     a = traj.unitcell_vectors[i]
#     b = np.asarray(a).reshape(-1)
#     header = 'Lattice="{0:.4f} {1:.4f} {2:.4f} {3:.4f} {4:.4f}, {5:.4f} {6:.4f} {7:.4f} {8:.4f}"' \
#                      .format(b[0], b[1], b[2], b[3], b[4], b[5], b[6], b[7], b[8]) \
#                      + ' ' + 'Properties=species:S:1:pos:R:3:force:R:3' + '  ' \
#                      + 'virial="0 0 0 0 0 0 0 0 0"' + '  '  \
#                      +  'energy= number' + ' ' + 'config_type=2H_300 pbc="T T T"'
#     f.write("{:}\n".format(header))
#     for j in range(len(frame)):
#         x, y, z = frame[j]*10
#         species_name = (topology.atom(j)).name
#         myfile.write("{}\t {}\t {}\t {}\t \n".format(x,y,z, force))
# myfile.close()